<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/Create_MobileNet_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import display, clear_output
import time
def sleepyTime():
    count=0
    try:
        while count <= 120:
            print(f'sleeping for {count} minutes')
            time.sleep(60)
            count+=1
    except KeyboardInterrupt:
        clear_output()
# sleepyTime()

In [ ]:
from __future__ import absolute_import
from IPython.display import display, clear_output
try:
    import tensorflow as tf
    print(f'tensorflow: {tf.__version__}')
except ModuleNotFoundError as err:
    %pip install tensorflow
    import tensorflow as tf
    print(f'\ntensorflow: {tf.__version__}')

from tensorflow import keras
from tensorflow.keras.preprocessing import image
from IPython.display import Image, display, clear_output
from tensorflow.keras.applications import imagenet_utils
import numpy as np

import glob, os, shutil, tarfile
from os.path import *
import numpy as np

contentPth = os.getcwd()
# deletePth = join(contentPth, 'sample_data')
if exists(join(contentPth, 'sample_data')):
    shutil.rmtree(join(contentPth, 'sample_data'))
    # deletePth=None
if contentPth == '/content':
    gdrivePth = join(contentPth, 'gdrive')
    myDrivePth = join(gdrivePth, 'My Drive')
    modelPth = join(myDrivePth, 'TensorflowModels')
    imgPth = join(myDrivePth, 'TensorflowImages')
    imgDirectoryPth = join(contentPth, 'Defcon_Images')
else:
    imgPth = join(contentPth, 'TensorflowImages')
    modelPth = join(contentPth, 'TensorflowModels')

try: import matplotlib.image as mpimg
except ModuleNotFoundError:
    %pip install matplotlib
    import matplotlib.image as mpimg
    
try:
    from google.colab import drive, files
    drive.flush_and_unmount()
    drive.mount('/content/gdrive', force_remount=True)
except: pass

try:
    os.chdir(myDrivePth)
    from BashColors import C
    from TarfileFunctions import tff
except Exception as err:
    # print(err)
    os.chdir(contentPth)
    from BashColors import C
    from TarfileFunctions import tff
    if not exists(imgPth):
        tff.listTarfiles()
        tff.extractTarfiles('Defcon_Images.tar.gz')
    else: print(f'{C.IBlue}{imgPth}{C.ColorOff} exists')
        
mobilenet_v2=None
imagePathList=[]
os.chdir(contentPth)
print(f'cwd: {C.Blue}{os.getcwd()}')

In [ ]:
be='''
if exists(imgPth):
    print(imgPth)
    os.chdir(imgPth)
    print(f'cwd: {C.IBlue}{os.getcwd()}')
    if not exists(imgDirectoryPth):
        shutil.copytree(imgPth, imgDirectoryPth)
'''

In [ ]:
# from tensorflow import keras
from tensorflow.keras import callbacks
early_stop = callbacks.EarlyStopping(
    patience=2,
    restore_best_weights=True)

save_callback = tf.keras.callbacks.ModelCheckpoint(
    'checkpoints/',
    save_weights_only=False,
    monitor='accuracy',
    save_best_only=True)

In [ ]:
def normalizeImage(set_image):
    print('Normalized')
    return tf.cast(set_image, dtype=tf.float32) / 255, labels

def augmentData(thisImage):
    print(thisImage)
    resize_rescale = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.Resizing(224, 224),
            tf.keras.layers.experimental.preprocessing.Rescaling(1.0/225) ])
    result = resize_rescale(thisImage)
    return result

In [ ]:
def showImage(thisImagePath, newWidth=64, newHeight=64):
    if exists(thisImagePath):
        image_name=basename(thisImagePath)
        # print(f'{C.IGreen}{thisImagePath}')
        print(f'{C.IWhite}{image_name}')

        thisImage = image.load_img(path=thisImagePath, 
                                       target_size=(newWidth, 
                                                    newHeight))
        imgArray = image.img_to_array(img=thisImage)
        imgArrayExpandDims = np.expand_dims(a=imgArray, axis=0)
        result = tf.keras.applications.mobilenet_v2.preprocess_input(imgArrayExpandDims)
        show = Image(filename=thisImagePath, width=newWidth,height=newHeight)
        display(show)

In [ ]:
imageFileListGlob=[]
os.chdir(imgPth)
imageFileListGlob=glob.glob('**', recursive=True)
print(len(imageFileListGlob))

for fil in sorted(imageFileListGlob):
    fullPath = abspath(fil)
    if isdir(fil):
        print(f'\n{C.IBlue}{fullPath}{C.ColorOff}')
    if not fullPath in imagePathList:
        imagePathList.append(fullPath)
    elif isfile(fil):
        try:
            showImage(thisImagePath=fullPath,
                      newWidth=128, newHeight=128)
        except: 
            print(f'{C.BRed}{abspath(fil)}{C.ColorOff}')
            imagePathList.remove(fullPath)
# clear_output()
print(len(imagePathList),'images')

In [ ]:
print(len(imagePathList))
print(len(imageFileListGlob))

for item in imageFileListGlob:
    if item.endswith('.png'):
        pass
    else: print(item)

In [ ]:
mobilenet_v2 = tf.keras.applications.MobileNetV2(input_shape=None,
                                                 alpha=1.0,
                                                 include_top=False,
                                                 weights=None,
                                                 input_tensor=None, 
                                                 pooling=None,
                                                 classes=1000,
                                                 classifier_activation='softmax')

mobilenet_v2.compile(optimizer='Adam')
# mobilenet_v2.summary()

modelName = mobilenet_v2.name + '_no_weights_no_top.h5'
modelSavePth = join(contentPth, modelName)

if exists(contentPth):
    mobilenet_v2.save(modelSavePth)
    print(f'saved: {C.Green}{modelSavePth}{C.ColorOff}')

In [ ]:
from tensorflow import keras
loadModelPth = join(contentPth, 'mobilenetv2_1.00_None_no_weights_no_top.h5')
re_model = keras.models.load_model(loadModelPth)
re_model.compile()
# re_model.summary()
re_model.fit()
re_model.fit( ,callbacks=['early_stop', 'save_callback'])

In [ ]:
configDict = re_model.get_config()

for item in configDict:
    print(f'{C.Blue}{item}:{C.ColorOff}')
    # print(configDict[item], '\n\n')

In [ ]:
try: import matplotlib.image as mpimg
except ModuleNotFoundError:
    %pip install matplotlib
    import matplotlib.image as mpimg

for newPath in sorted(imagePathList):
    if isfile(newPath) and newPath.endswith('.png'):
        showImage(newPath, newWidth=64, newHeight=64)
        img = image.load_img(path=newPath, target_size=(224,244))
        imageArray = image.img_to_array(img)
        imageArrayExpandDims = np.expand_dims(imageArray, axis=0)
    else:
        imagePathList.remove(newPath)
    

In [ ]:
os.chdir(imgDirectoryPth)

# tarfilePth=''
finder=glob.glob('**', recursive=True)
for fil in sorted(finder):
    tarfilePth=abspath(fil)
    print(tarfilePth)

In [ ]:
os.chdir(contentPth)
os.chdir(myDrivePth)
# print(tarfilePth)

tar = tarfile.open(name='GDrive_Defcon_Images.tar.gz',
                   mode='w:gz',
                   fileobj=None,
                   bufsize=10240)
tar.add(name=imgDirectoryPth,
        arcname=None,
        recursive=True,
        filter=None)



In [ ]:
help(tarfile.open)